In [7]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
batchsize=100
training_data = datasets.FashionMNIST(root="../fashion_mnist", train=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="../fashion_mnist", train=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(training_data, batch_size=batchsize)
test_dataloader = DataLoader(test_data, batch_size=batchsize)

In [4]:
# define hyperparmaters
sequence_len = 28
input_len = 28
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 5
learning_rate = 0.01

In [14]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes)
    
    def forward(self, X):
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out, _ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out

In [15]:
model = LSTM(input_len, hidden_size, num_classes, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [19]:
loss_func = nn.CrossEntropyLoss()
sgd = optim.SGD(model.parameters(), lr=learning_rate)
adam = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
def train(num_epochs, model, train_dataloader, loss_func, optimizer):
    total_steps = len(train_dataloader)

    for epoch in range(num_epochs):
        for batch, (images, labels) in enumerate(train_dataloader):
            images = images.reshape(-1, sequence_len, input_len)
            
            output = model(images)
            loss = loss_func(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1)%100 == 0:
                print(f"Epoch: {epoch+1}; Batch {batch+1} / {total_steps}; Loss: {loss.item():>4f}")

In [22]:
train(num_epochs, model, train_dataloader, loss_func, adam)

Epoch: 1; Batch 100 / 600; Loss: 0.918076
Epoch: 1; Batch 200 / 600; Loss: 0.766438
Epoch: 1; Batch 300 / 600; Loss: 0.943084
Epoch: 1; Batch 400 / 600; Loss: 0.604074
Epoch: 1; Batch 500 / 600; Loss: 0.645839
Epoch: 1; Batch 600 / 600; Loss: 0.364988
Epoch: 2; Batch 100 / 600; Loss: 0.450433
Epoch: 2; Batch 200 / 600; Loss: 0.420381
Epoch: 2; Batch 300 / 600; Loss: 0.430095
Epoch: 2; Batch 400 / 600; Loss: 0.358064
Epoch: 2; Batch 500 / 600; Loss: 0.502615
Epoch: 2; Batch 600 / 600; Loss: 0.271195
Epoch: 3; Batch 100 / 600; Loss: 0.339103
Epoch: 3; Batch 200 / 600; Loss: 0.349369
Epoch: 3; Batch 300 / 600; Loss: 0.318679
Epoch: 3; Batch 400 / 600; Loss: 0.331628
Epoch: 3; Batch 500 / 600; Loss: 0.415435
Epoch: 3; Batch 600 / 600; Loss: 0.260229
Epoch: 4; Batch 100 / 600; Loss: 0.357943
Epoch: 4; Batch 200 / 600; Loss: 0.296318
Epoch: 4; Batch 300 / 600; Loss: 0.304155
Epoch: 4; Batch 400 / 600; Loss: 0.296495
Epoch: 4; Batch 500 / 600; Loss: 0.378561
Epoch: 4; Batch 600 / 600; Loss: 0

In [24]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [25]:
test_output = model(test_images.view(-1, 28, 28))

In [28]:
predicted = torch.max(test_output, 1)[1]
predicted

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 2, 1, 2, 4, 8, 0, 2, 5, 7, 5,
        1, 2, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 0, 1, 0, 9, 6, 7, 2, 1,
        4, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 0, 3, 7, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 2, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [29]:
correct = [1 for i in range(100) if predicted[i] == test_labels[i]]

In [30]:
percentage_correct = sum(correct)/100

In [31]:
percentage_correct

0.9

In [34]:
def test_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # reshape images
            X = X.reshape(-1, 28, 28)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    return 100*correct

In [35]:
test_loop(test_dataloader, model, loss_func, adam)

Test Error:
 Accuracy: 87.5%, Avg loss: 0.346195



87.52